In [6]:
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset_builder
from datasets import load_dataset
import numpy as np
import evaluate
import torch

In [7]:
tokenizer = AutoTokenizer.from_pretrained("KETI-AIR-Downstream/long-ke-t5-base-translation-aihub-ko2en",model_max_length=512)
model = AutoModelForSeq2SeqLM.from_pretrained("KETI-AIR-Downstream/long-ke-t5-base-translation-aihub-ko2en")

In [8]:
#source = """translate_ko2en: 그럼...그...갑작스럽간한데..."""

In [9]:
"""
input_ids = tokenizer(source, return_tensors="pt").input_ids
gen_seq = model.generate(
    input_ids,
    num_beams=4,
    max_length=1024
)
print(tokenizer.decode(gen_seq[0], skip_special_tokens=True))
"""

'\ninput_ids = tokenizer(source, return_tensors="pt").input_ids\ngen_seq = model.generate(\n    input_ids,\n    num_beams=4,\n    max_length=1024\n)\nprint(tokenizer.decode(gen_seq[0], skip_special_tokens=True))\n'

In [12]:
train  = load_dataset("Moo/korean-parallel-corpora", split="train")
test = load_dataset("Moo/korean-parallel-corpora", split="test")
#validation = load_dataset("Moo/korean-parallel-corpora", split="validation")

In [50]:
train[0]

{'ko': '개인용 컴퓨터 사용의 상당 부분은 "이것보다 뛰어날 수 있느냐?"',
 'en': 'Much of personal computing is about "can you top this?"'}

In [13]:
#train_encodings = tokenizer(train['ko'], truncation=True, padding=True)

In [15]:
train_encodings = tokenizer(train['ko'], text_target=train['en'], truncation=True, padding=True)

In [16]:
test_encodings = tokenizer(test['ko'], text_target=test['en'], truncation=True, padding=True)

In [17]:
source_lang = "ko"
target_lang = "en"
prefix = "translate Korean to English: "

In [18]:
def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [19]:
#tokenized_books = train.map(preprocess_function, batched=True)

In [34]:
class Dataset(torch.utils.data.Dataset):    
    def __init__(self, encodings, labels=None):          
        self.encodings = encodings        
        self.labels = labels
     
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

In [43]:
class my_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        print(item)
        return item

    def __len__(self):
        return len(self.labels)

In [44]:
dataset_train = Dataset(train_encodings,train['en'])

In [45]:
dataset_test= Dataset(test_encodings,test['en'])

In [46]:
dataset_train2 = my_Dataset(train,train.features)
dataset_test2= my_Dataset(test,train.features)

In [29]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [51]:
from transformers import TrainingArguments, Trainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    fp16=True,
    remove_unused_columns=False,
    max_steps=10000,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train2,
    eval_dataset=dataset_test2,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


import transformers
transformers.logging.set_verbosity_info()

trainer.train()

Found safetensors installation, but --save_safetensors=False. Safetensors should be a preferred weights saving format due to security and performance reasons. If your model cannot be saved by safetensors please feel free to open an issue at https://github.com/huggingface/safetensors!
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
max_steps is given, it will override any value given in num_train_epochs
***** Running training *****
  Num examples = 2
  Num Epochs = 10,000
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 10,000
  Number of trainable parameters = 296,696,448


AttributeError: 'Dataset' object has no attribute 'items'